In [1]:
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import reset_seeds


reset_seeds(seed_value=39)
TIMESTEMPS = 800
FM = 20000
FM_TARGET = 2000
dataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelFreqIDs = ["mlp","svr","linearSGD"]
modelTimeIDs = ["seq2point","lstm","esn","rocket","tcn"]
params = {}

dataFreq = featureExtraction(dataID,statorFreqs=[37],testsID=[21,24])  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataFreqTest = featureExtraction(dataID,statorFreqs=[37],testsID=[21,24,27],scaler_params=dataFreq.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27

dataTime = featureExtraction(dataID,statorFreqs=[37],
                            testsID=[21,24],
                            featsDomain="time",
                            timesteps=TIMESTEMPS,
                            Fm= FM,
                            Fm_target=FM_TARGET)  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27

dataTimeTest = featureExtraction(dataID,
                                statorFreqs=[37],
                                testsID=[21,24,27],
                                scaler_params=dataTime.get_scaler_params(),
                                featsDomain="time",
                                timesteps=TIMESTEMPS,
                                Fm= FM,
                                Fm_target=FM_TARGET)  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27


MODELS = []
X_test = []
Y_test = []

for modelID in modelFreqIDs:
    model = modelGenerator(modelID=modelID, data=dataFreq,params=params)
    model.train()
    MODELS.append(model)
    # Freq-based
    X_freq_test = dataFreqTest.X
    y_freq_test = dataFreqTest.y
    # Selecting temps higher than 100ºC
    idx = np.argwhere(y_freq_test>100)[:,0]
    Y_test.append(y_freq_test[idx])
    X_test.append(X_freq_test[idx,...])

for modelID in modelTimeIDs:
    model = modelGenerator(modelID=modelID, data=dataTime,params=params)
    model.train()
    MODELS.append(model)
    X_time_test = dataTimeTest.X
    y_time_test = dataTimeTest.y[:,-1]
    # Selecting temps higher than 100ºC
    idx = np.argwhere(y_time_test>100)[:,0]
    Y_test.append(y_time_test[idx])
    X_test.append(X_time_test[idx,...])

modelIDs = modelFreqIDs+modelTimeIDs


2023-10-03 21:40:20.284206: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 6)]               0         
                                                                 
 dense_0 (Dense)             (None, 130)               910       
                                                                 
 dense_1 (Dense)             (None, 140)               18340     
                                                                 
 dense (Dense)               (None, 1)                 141       
                                                                 
Total params: 19,391
Trainable params: 19,391
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300


2023-10-03 21:41:02.495440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 21:41:02.753645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22311 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


217/625 [=========>....................] - ETA: 0s - loss: 10135.8008 - mean_squared_error: 10135.8008 - mean_absolute_error: 95.0070 

2023-10-03 21:41:03.375078: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


625/625 [==============================] - 1s 888us/step - loss: 5666.9331 - mean_squared_error: 5666.9331 - mean_absolute_error: 64.9100 - val_loss: 2297.3025 - val_mean_squared_error: 2297.3025 - val_mean_absolute_error: 43.8901
Epoch 2/300
625/625 [==============================] - 0s 774us/step - loss: 690.0974 - mean_squared_error: 690.0974 - mean_absolute_error: 19.3038 - val_loss: 305.8610 - val_mean_squared_error: 305.8610 - val_mean_absolute_error: 14.0646
Epoch 3/300
625/625 [==============================] - 0s 762us/step - loss: 79.6833 - mean_squared_error: 79.6833 - mean_absolute_error: 5.9502 - val_loss: 42.7621 - val_mean_squared_error: 42.7621 - val_mean_absolute_error: 5.1292
Epoch 4/300
625/625 [==============================] - 0s 772us/step - loss: 15.7468 - mean_squared_error: 15.7468 - mean_absolute_error: 2.8678 - val_loss: 14.4375 - val_mean_squared_error: 14.4375 - val_mean_absolute_error: 3.0616
Epoch 5/300
625/625 [==============================] - 0s 765us/

/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
2023-10-03 21:41:12.811741: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


103/103 [==============================] - 2s 4ms/step - loss: 1707.0441 - mean_squared_error: 1707.0441 - mean_absolute_error: 32.1870 - val_loss: 1200.4432 - val_mean_squared_error: 1200.4432 - val_mean_absolute_error: 31.3873
Epoch 2/300
103/103 [==============================] - 0s 2ms/step - loss: 940.4968 - mean_squared_error: 940.4968 - mean_absolute_error: 26.3946 - val_loss: 1078.4381 - val_mean_squared_error: 1078.4381 - val_mean_absolute_error: 29.6834
Epoch 3/300
103/103 [==============================] - 0s 2ms/step - loss: 739.7314 - mean_squared_error: 739.7314 - mean_absolute_error: 22.8116 - val_loss: 176.6044 - val_mean_squared_error: 176.6044 - val_mean_absolute_error: 10.0546
Epoch 4/300
103/103 [==============================] - 0s 2ms/step - loss: 83.6524 - mean_squared_error: 83.6524 - mean_absolute_error: 6.8297 - val_loss: 26.2396 - val_mean_squared_error: 26.2396 - val_mean_absolute_error: 4.2341
Epoch 5/300
103/103 [==============================] - 0s 2ms/st

In [2]:
modelIDs = modelFreqIDs+modelTimeIDs
# Estimations
Y_test_est = []
for ii,model in enumerate(MODELS):
    Y_test_est.append(model.predict(X_test[ii]))


22/22 [==============================] - 0s 535us/step


21/21 [==============================] - 0s 9ms/step
Getting  states
Getting outputs
21/21 [==============================] - 2s 61ms/step


In [15]:
# Guardado de las estimaciones en json
import address
import os
import json
estimations = {}
for ii,model in enumerate(modelIDs):
    estimations[model]={}
    estimations[model]['y_test'] = Y_test[ii].tolist()
    estimations[model]['y_test_est'] = Y_test_est[ii].tolist()

with open(os.path.join(address.path_results_metrics,"estimations_res.json"),'w') as fp:
    json.dump(estimations, fp)

In [20]:
# # Ejemplo de lectura
# with open(os.path.join(address.path_results_metrics,"estimations_res.json"),'r') as fp:
#     estimations_ = json.load(fp)

# for k,v in estimations_.items():
#     estimations[k]['y_test_est']=np.array(estimations[k]['y_test_est'])
#     estimations[k]['y_test']=np.array(estimations[k]['y_test'])


array([101.291, 101.387, 101.479, ..., 166.781, 166.644, 166.495])

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()

MODEL_LABELS = {"mlp": "MLP",
                "svr": "SVR",
                "linearSGD":"MLR",
                "lstm": "LSTM",
                "rocket": "rocket",
                "seq2point":"CNN",
                "tcn": "TCN",
                "esn":"ESN"}

for ii,model in enumerate(MODELS):
    p = figure(title=f"Residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=350)
    y_test = Y_test[ii]
    y_test_est = Y_test_est[ii]
    p.line(np.arange(len(y_test)),y_test, line_width=2,line_alpha=0.4, line_color='gray', legend_label="Ground truth" )
    p.line(np.arange(len(y_test)),y_test_est, line_width=2, line_alpha=0.8,line_color='green',legend_label="Estimation")
    p.legend.location = 'bottom_right'
    p.yaxis.axis_label = r"T (ºC)"
    p.xaxis.axis_label = r"sample"
    show(p)
    

Loading BokehJS ...

In [8]:
# Best models freq and time 

#SEL_MODELS = ["mlp","svr","seq2point","lstm"]
SEL_MODELS = ["mlp"]
colors = ["green",'orange']
p = figure(title=f"Temperature monitoring",width=900,height=350)

for ii,model in enumerate(SEL_MODELS):
    jj = modelIDs.index(model)
    y_test = Y_test[jj]
    y_test_est = Y_test_est[jj]    
    p.line(np.arange(len(y_test)),y_test, line_width=2,line_alpha=0.4, line_color='gray', legend_label="Ground truth")
    p.line(np.arange(len(y_test)),y_test_est, line_width=2, line_color=colors[ii],line_alpha=0.6,legend_label=f"{MODEL_LABELS[modelIDs[jj]]}")
    p.legend.location = 'bottom_right'
    p.yaxis.axis_label = r"T (ºC)"
    p.xaxis.axis_label = r"sample"
show(p)


from scipy import signal

b = signal.firwin(151,0.01)

r = figure(title=f"Residuals",width=900,height=250,y_range=(0,50))

for ii,model in enumerate(SEL_MODELS):
    jj = modelIDs.index(model)
    y_test = Y_test[jj]
    y_test_est = Y_test_est[jj] 
    residuals = np.square(y_test-y_test_est)
    # Removing impulsional noise
    residual_med = signal.medfilt(residuals)
    res_med = residuals - residual_med
    residuals = np.delete(residuals,res_med>40)
    residuals_ = signal.filtfilt(b, 1, residuals)
    
    r.line(np.arange(len(y_test)),residuals, line_width=2, line_alpha=0.3,line_color='gray',legend_label="normal")
    r.line(np.arange(len(y_test)),residuals_, line_width=2, line_alpha=0.8,line_color='blue',legend_label="filtered")
    r.legend.location = 'top_right'
    r.yaxis.axis_label = r"$$\|T-\hat{T}\|^2$$"
    r.xaxis.axis_label = r"sample"
show(r)



In [9]:
SEL_MODELS = ["lstm"]
colors = ["green",'orange']
p = figure(title=f"Temperature monitoring",width=900,height=350)

for ii,model in enumerate(SEL_MODELS):
    jj = modelIDs.index(model)
    y_test = Y_test[jj]
    y_test_est = Y_test_est[jj]    
    p.line(np.arange(len(y_test)),y_test, line_width=2,line_alpha=0.4, line_color='gray', legend_label="Ground truth")
    p.line(np.arange(len(y_test)),y_test_est, line_width=2, line_color=colors[ii],line_alpha=0.6,legend_label=f"{MODEL_LABELS[modelIDs[jj]]}")
    p.legend.location = 'bottom_right'
    p.yaxis.axis_label = r"T (ºC)"
    p.xaxis.axis_label = r"sample"
show(p)

b = signal.firwin(151,0.01)

r = figure(title=f"Residuals",width=900,height=250,y_range=(0,50))

for ii,model in enumerate(SEL_MODELS):
    jj = modelIDs.index(model)
    y_test = Y_test[jj]
    y_test_est = Y_test_est[jj] 
    residuals = np.square(y_test-y_test_est)
    # Removing impulsional noise
    residual_med = signal.medfilt(residuals)
    res_med = residuals - residual_med
    residuals = np.delete(residuals,res_med>40)
    residuals_ = signal.filtfilt(b, 1, residuals)
    
    r.line(np.arange(len(y_test)),residuals, line_width=2, line_alpha=0.3,line_color='gray',legend_label="normal")
    r.line(np.arange(len(y_test)),residuals_, line_width=2, line_alpha=0.8,line_color='blue',legend_label="filtered")
    r.legend.location = 'top_right'
    r.yaxis.axis_label = r"$$\|T-\hat{T}\|^2$$"
    r.xaxis.axis_label = r"sample"
show(r)

In [2]:
modelIDs

NameError: name 'modelIDs' is not defined

In [4]:
from scipy import signal

b = signal.firwin(151,0.01)
RESIDUALS = []
for ii,model in enumerate(MODELS):
    y_test = Y_test[ii]
    y_test_est = Y_test_est[ii]
    residuals = np.square(y_test-y_test_est)
    # Removing impulsional noise
    residual_med = signal.medfilt(residuals)
    res_med = residuals - residual_med
    residuals = np.delete(residuals,res_med>40)
    residuals_ = signal.filtfilt(b, 1, residuals)
    p = figure(title=f"Abs residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=250,y_range=(0,50))
    p.line(np.arange(len(y_test)),residuals, line_width=2, line_alpha=0.3,line_color='gray',legend_label="normal")
    p.line(np.arange(len(y_test)),residuals_, line_width=2, line_alpha=0.8,line_color='blue',legend_label="filtered")
    p.line(np.arange(len(y_test)),res_med, line_width=2, line_alpha=0.8,line_color='orange',legend_label="filtered med")
    p.legend.location = 'top_right'
    p.yaxis.axis_label = r"$$|T-\hat{T}|^2_2$$"
    p.xaxis.axis_label = r"sample"
    show(p)
    RESIDUALS.append(residuals_)


## Boxplot residuals
Descriptive statisdics of the residuals obtained from applying all the models to all the avilable tests (#21, #24 and #27).  

In [3]:
#Boxplox residuals
from bokeh.models import ColumnDataSource, Whisker
from itertools import product

testIDs = [21,24,27]
RESIDUALDATA = []

for test, model in product(testIDs[::-1],modelIDs):
    if model in modelFreqIDs:
        dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[test],scaler_params=dataFreq.get_scaler_params(),featsDomain="freq")  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
        X_test = dataTest.X
        y_test = dataTest.y
    else:
        dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[test],scaler_params=dataTime.get_scaler_params(),featsDomain="time",timesteps=800,Fm= 20000,Fm_target=2000)  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
        X_test = dataTest.X
        y_test = dataTest.y[:,-1]


    # Selecting temps higher than 100ºC
    idx = np.argwhere(y_test>100)[:,0]
    y_test = y_test[idx]
    X_test = X_test[idx,...]
    idx_model = modelIDs.index(model)
    RESIDUALDATA.append({"model": MODELS[idx_model],"data":(X_test,y_test) })

# Get the residuals from all the tests

In [14]:
from scipy import signal
b = signal.firwin(30,0.01)
RESIDUALS = []
RESIDUALS_SIGN = []
maxRes = {}
for ii,(test,model) in enumerate(product(testIDs[::-1],modelIDs)):
    print(f"Test: {test} and model {model}  ")
    modelObj = RESIDUALDATA[ii]["model"]
    X_test,y_test   = RESIDUALDATA[ii]["data"]
    y_test_est      = modelObj.predict(X_test)

    residuals = np.square(y_test-y_test_est)
    residual_med = signal.medfilt(residuals)
    res_med = residuals - residual_med
    residuals = np.delete(residuals,res_med>15)
    residuals_ = signal.filtfilt(b, 1, residuals)
    
    if test ==27:
        maxRes[model] = residuals_.max()
    

    residuals_sign = y_test-y_test_est
    residual_med_sign = signal.medfilt(residuals_sign)
    residual_med_sign = residuals_sign - residual_med_sign
    residuals_sign = np.delete(residuals_sign,residual_med_sign>5)
    #residuals_ = signal.filtfilt(b, 1, residuals)


    #residuals_ = residuals_/maxRes[model]
    df_residuals = pd.DataFrame(residuals_,columns=['Residuals'])
    df_residuals_sign = pd.DataFrame(residuals_sign,columns=['Residuals'])
    df_residuals["Test"] = test
    df_residuals["Model"] = model
    df_residuals_sign["Test"] = test
    df_residuals_sign["Model"] = model
    RESIDUALS.append(df_residuals)
    RESIDUALS_SIGN.append(df_residuals_sign)

RESIDUALS = pd.concat(RESIDUALS,axis=0)
RESIDUALS_SIGN = pd.concat(RESIDUALS_SIGN,axis=0)

    

Test: 27 and model mlp  
8/8 [==============================] - 0s 587us/step


Test: 27 and model svr  
Test: 27 and model linearSGD  
Test: 27 and model seq2point  
8/8 [==============================] - 0s 2ms/step
Test: 27 and model lstm  
8/8 [==============================] - 0s 7ms/step
Test: 27 and model esn  
Getting  states
Getting outputs
Test: 27 and model rocket  
Test: 27 and model tcn  
8/8 [==============================] - 0s 46ms/step
Test: 24 and model mlp  
8/8 [==============================] - 0s 537us/step
Test: 24 and model svr  
Test: 24 and model linearSGD  
Test: 24 and model seq2point  
8/8 [==============================] - 0s 2ms/step
Test: 24 and model lstm  
8/8 [==============================] - 0s 6ms/step
Test: 24 and model esn  
Getting  states
Getting outputs
Test: 24 and model rocket  
Test: 24 and model tcn  
8/8 [==============================] - 0s 46ms/step
Test: 21 and model mlp  
6/6 [==============================] - 0s 606us/step
Test: 21 and model svr  
Test: 21 and model linearSGD  
Test: 21 and model seq2point  
6/6

In [15]:
import os
import address
RESIDUALS.to_csv(os.path.join(address.path_results_metrics,"residuals.csv"),index=False)
RESIDUALS_SIGN.to_csv(os.path.join(address.path_results_metrics,"residuals_sign.csv"),index=False)

In [35]:
from bokeh.models import ColumnDataSource, Whisker
#Extracting quartiles
Q_df = RESIDUALS.groupby(["Test","Model"]).quantile([0.25,0.5,0.75])
Q_df = Q_df.unstack().reset_index()
Q_df.columns = ["Test","Model","q1","q2","q3"]

# compute IQR 
iqr = Q_df.q3 - Q_df.q1
Q_df["upper"] = Q_df.q3 + 1.5*iqr
Q_df["lower"] = Q_df.q1 - 1.5*iqr

#Figure params 
colors = ["#BB8FCE", "#82E0AA","#F0B27A"]

p = figure( y_range=[-4.5,12], width = 900, height = 300,
           title="Residuals per test",
           y_axis_label="residuals values")
for i,test in enumerate(testIDs):
    df = Q_df[Q_df.Test == test].copy()
    df.index = df.Model; df.index.name=None; df = df.loc[modelIDs]
    df["ModelLabel"] = pd.Categorical(df["Model"],categories=modelIDs).codes*4
    df.ModelLabel = df.ModelLabel + 0.7*i
    #df = df.reindex(modelIDs)
    source = ColumnDataSource(df)
    whisker = Whisker(base="ModelLabel", upper="upper", lower="lower", line_color="#626567", source=source)
    p.add_layout(whisker)
    p.vbar("ModelLabel", 0.6, "q2", "q3", source=source, legend_label = f"#{test}",  line_color="#626567",color=colors[i])
    p.vbar("ModelLabel", 0.6, "q1", "q2", source=source,  line_color="#626567",color=colors[i])
ticks = list(range(1,len(MODELS)*4,4))
model_ticks = [MODEL_LABELS[m] for m in modelIDs ]
p.xaxis.ticker = ticks
p.xaxis.major_label_overrides = dict(zip(ticks,model_ticks))
#p.legend.location=
p.add_layout(p.legend[0], 'right')
show(p)

In [37]:

#Extracting quartiles
Q_df = RESIDUALS.groupby(["Test","Model"]).quantile([0.25,0.5,0.75])
Q_df = Q_df.unstack().reset_index()
Q_df.columns = ["Test","Model","q1","q2","q3"]
selModelIDs = ["lstm","seq2point","mlp","svr"]
Q_df = Q_df.loc[Q_df["Model"].isin(selModelIDs)]


# compute IQR 
iqr = Q_df.q3 - Q_df.q1
Q_df["upper"] = Q_df.q3 + 1.5*iqr
Q_df["lower"] = Q_df.q1 - 1.5*iqr

#Figure params 
colors = ["#BB8FCE", "#82E0AA","#F0B27A"]

p = figure( y_range=[-4,7.5],  width = 900, height = 300,
           title="Residuals per test",
           y_axis_label="residuals values")
for i,test in enumerate(testIDs):
    df = Q_df[Q_df.Test == test].copy()
    df.index = df.Model; df.index.name=None; df = df.loc[selModelIDs]
    df["ModelLabel"] = pd.Categorical(df["Model"],categories=selModelIDs).codes*4
    df.ModelLabel = df.ModelLabel + 0.7*i
    #df = df.reindex(modelIDs)
    source = ColumnDataSource(df)
    whisker = Whisker(base="ModelLabel", upper="upper", lower="lower", line_color="#626567", source=source)
    p.add_layout(whisker)
    p.vbar("ModelLabel", 0.6, "q2", "q3", source=source, legend_label = f"#{test}",  line_color="#626567",color=colors[i])
    p.vbar("ModelLabel", 0.6, "q1", "q2", source=source,  line_color="#626567",color=colors[i])
ticks = list(range(1,len(selModelIDs)*4,4))
model_ticks = [MODEL_LABELS[m] for m in selModelIDs ]
p.xaxis.ticker = ticks
p.xaxis.major_label_overrides = dict(zip(ticks,model_ticks))
#p.legend.location=
p.add_layout(p.legend[0], 'right')
show(p)

: 

In [28]:

Q_df

,Test,Model,q1,q2,q3,upper,lower
2,21,lstm,0.389076,0.619696,1.146031,2.281464,-0.746357
3,21,mlp,0.359185,0.474811,0.668421,1.132276,-0.104669
5,21,seq2point,0.375274,0.487472,0.660979,1.089537,-0.053285
10,24,lstm,0.396266,0.531783,0.693700,1.139851,-0.049884
11,24,mlp,0.377144,0.478219,0.656691,1.076010,-0.042175
13,24,seq2point,0.423802,0.516794,0.626338,0.930143,0.119997
18,27,lstm,0.485413,0.705957,1.451117,2.899673,-0.963143
19,27,mlp,0.508052,0.740938,1.529059,3.060569,-1.023459
21,27,seq2point,0.449822,0.556709,0.917165,1.618179,-0.251193
